In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, f1_score
from pathlib import Path

current_path = Path().resolve().parent
print(current_path)

In [2]:
def parse_file(file_path):
    instances = []
    try:
        with open(file_path, encoding='utf-8') as nf:
            lines = nf.readlines()
            tokens, labels, predictions = [], [], []
            for line in lines:
                if line.strip():
                    tk, lb, pr = line.split()
                    tokens.append(tk)
                    labels.append(lb)
                    predictions.append(pr)
                else:
                    instances.append((tokens, labels, predictions))
                    tokens, labels, predictions = [], [], []
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except ValueError:
        print(f"Invalid file format: {file_path}")

    return instances

def get_best_run_number(row):
    columns = [f'run{i}' for i in range(10)]
    values = row[columns].values
    return np.argmax(values)

In [3]:
# Since the tokenizer may divide each token into two or more subtokens, the predictions may contain subtokens.
# Subtokens' labels are equal to the label of parent token.
# We must join subtokens with their parent tokens for obtaining the same amount of tokens as original dataset.
def format_instances(tokens, labels, predictions):
    output_tokens, output_labels, output_predictions = [], [], []
    for instance_tokens, instance_labels, instance_predictions in zip(tokens, labels, predictions):

        clean_tokens = []
        clean_labels = []
        clean_preds = []
        current_parent_token = ""
        parent_token_label = '0'
        parent_token_pred = '0'

        for t, l, p in zip(instance_tokens, instance_labels, instance_predictions):
            if t.startswith("##"):
                current_parent_token += t[2:]
            else:
                if current_parent_token != "":
                    clean_tokens.append(current_parent_token)
                    clean_labels.append(parent_token_label)
                    clean_preds.append(parent_token_pred)

                current_parent_token = t
                parent_token_label = l
                parent_token_pred = p

        output_tokens.append(clean_tokens)
        output_labels.append(clean_labels)
        output_predictions.append(clean_preds)

    return output_tokens, output_labels, output_predictions

In [4]:
f1_results = pd.read_csv(str(current_path / 'data.csv'))
f1_results['best_run'] = f1_results.apply(lambda row: get_best_run_number(row), axis = 1)
# print(f1_results.shape)
# f1_results.head()

def get_labels_and_predictions(df, elem, test, model):

    best_run = df.loc[(df['train'] == elem) & (df['test'] == test) & (df['model'] == model)].best_run.values[0]
    
    folder_dir = f"{elem}_{model}"
    model_name = "-".join(model.split("_"))
    file_name = f"results-{elem}-{test}-{model_name}-{best_run}.txt"
    file_path = (current_path / 'results' / 'BIO' / folder_dir / file_name)
    
    instances = parse_file(file_path)

    tokens = [x[0] for x in instances]
    labels = [x[1] for x in instances]
    predictions = [x[2] for x in instances]
    
    new_tokens, new_labels, new_predictions = format_instances(tokens, labels, predictions)
    
    # Task 1: Replace '0' with 'O', '1' with 'B', and '2' with 'I'
    labels = [['O' if x == '0' else 'B' if x == '1' else 'I' for x in inner_list] for inner_list in new_labels]
    predictions = [['O' if x == '0' else 'B' if x == '1' else 'I' for x in inner_list] for inner_list in new_predictions]
    
    # Task 2: Flatten the list of lists
    flattened_labels = [label for inner_list in labels for label in inner_list]
    flattened_predictions = [label for inner_list in predictions for label in inner_list]
    
    return flattened_labels, flattened_predictions

In [ ]:

labels1, predictions1 = get_labels_and_predictions(f1_results, 'pe', 'pe', 'bert_crf')
labels2, predictions2 = get_labels_and_predictions(f1_results, 'we', 'we', 'bert_crf')
labels3, predictions3 = get_labels_and_predictions(f1_results, 'abam', 'abam', 'bert_crf')

# Function to plot a confusion matrix
def plot_confusion_matrix(ax, labels_true, labels_pred, title):
    cm = confusion_matrix(labels_true, labels_pred)

    print(cm)

    cm_row_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    cm_overall_norm = cm.astype('float') / cm.sum() * 100
    sns.heatmap(cm_row_norm, annot=True, fmt=".1f", cmap="Blues", ax=ax, cbar = False, annot_kws={"size": 14})
    ax.set_title(title)
    ax.set_xlabel('Predicho')
    ax.set_xticklabels(['B', 'I', 'O'])
    ax.set_yticklabels(['B', 'I', 'O'])
    ax.set_ylabel('Correcto')

# # Create subplots for three confusion matrices in a row
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# # Plot each confusion matrix
plot_confusion_matrix(axes[0], labels1, predictions1, 'PE')
plot_confusion_matrix(axes[1], labels2, predictions2, 'WE')
plot_confusion_matrix(axes[2], labels3, predictions3, 'ABAM')

# # Adjust layout and display the plot
plt.tight_layout()

# plt.savefig(str((current_path) / 'figures' / 'ch3-confugsionMatrices.pdf'), format='pdf')

plt.show()
